# Session 3

### Goals:

1. Generalize previous codebase to multiple data points
2. Introduce torch.nn Module
3. Introduce optimizers
4. Train a linear classifier
5. Train a FFNN

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from IPython import display

import seaborn as sns
import torch

In [ ]:
torch.manual_seed(42)

In [ ]:
def viz_1d(X, Y, title = None):
    
    if isinstance(Y, torch.Tensor):
        Y = Y.detach()
        
    plt.figure(figsize=(14, 10))
    if title:
        plt.title(title)
    plt.scatter(X, Y)
    plt.show()

def viz_pred(X, Y, Y_, title=None):
    
    if isinstance(Y_, torch.Tensor):
        Y_ = Y_.detach()
        
    plt.figure(figsize=(14, 10))
    if title:
        plt.title(title)
        
    plt.plot(X, Y, 'o')
    plt.plot(X, Y_, '+')
#     plt.scatter(X, Y, color='red')
    plt.show()
    
def viz_all(model, inputs,outputs= None, predictions= None,  ax=None ):
    inputs_scale = (inputs.max()-inputs.min())/10.0
    _X = torch.tensor(np.mgrid[inputs.min()-inputs_scale: inputs.max()+inputs_scale:0.1])
    with torch.no_grad():
        _Y = model(_X).detach()
        
    if ax is None: 
        fig = plt.figure(figsize=(14, 8), dpi=100)
        ax = fig.add_subplot(111)
        old_ax = False
    else:
        old_ax = True
    
    if outputs is not None:
        ax.scatter(inputs, outputs,  c='b', marker="s", label='Y Gold')
    
    if predictions is not None:
        ax.scatter(inputs, predictions, c='r', marker='+', label='Y Pred')
    
    ax.plot(_X, _Y, label='model')
    if not old_ax:
        plt.legend(loc='upper left')
        plt.show()
    

In [ ]:

def true_fn(x):
#     return float(x**2 - (x) + 3)
    return float(3*x + 10)

n_samples = 200
X = torch.randint(-400, 400, (n_samples,))
Y = [true_fn(x) for x in X]
Y = torch.tensor(Y)
fx = lambda x : (m*x) + c
viz_1d(X, Y)

In [ ]:
# fx = lambda x : (m*x) + c


class LinClf(torch.nn.Module):
    
    def __init__(self,):
        # we declare the params here
        super().__init__()
        
        self.m = torch.nn.Parameter(torch.tensor(3.5))
        self.c = torch.nn.Parameter(torch.tensor(9.0))
        
    def forward(self, inputs):
        # we define 'what to do with params here'
        return (inputs*self.m) + self.c
    

In [ ]:
class TwoOrderClf(torch.nn.Module):
    ...

In [ ]:
# Make model
m = LinClf()

In [ ]:
#  and loss fn
lfn = torch.nn.MSELoss(reduction='mean')

In [ ]:
viz_all(m, X, Y)

## Why model class?

- Saving/Loading from Disk
- Setting training mode/evaluation mode
- Easy access to all parameters
- Structure is v. important for complex models
- That is 'pytorch' way!

In [ ]:
# Make optimizer
opt = torch.optim.SGD(m.parameters(), lr=0.00005)

In [ ]:
# See optimizer things (param groups)
opt.param_groups

In [ ]:
m, lfn, opt

In [ ]:
epochs = 2000

In [ ]:

fig = plt.figure(figsize = (10,4))
ax1 = fig.add_subplot(111)
# xlim = (-1,2, 1.2)
# ylim = (-1, 14)
# ax1.set_xlim(-1.2, 1.2)
# ax1.set_ylim(-1, 14)
losses = []

# our dataset = X, Y

for e in range(epochs):

    opt.zero_grad() 
    
    # Lets sample a batch
    Xrange = torch.randint(0, n_samples, (20, ))
    _X = X[Xrange]
    _Y = Y[Xrange]
    
    # Make model predictions
    Y_pred = m(_X)
    
    # Calculate Loss
    loss = lfn(Y_pred, _Y)
    
#     print(f"Parameters before update:\n\ta1: {m.a1.item()}\tgrad: {m.a1.grad.item() if m.a1.grad else None}\n\ta2: {m.a2.item()}\tgrad: {m.a2.grad.item() if m.a2.grad else None}\n\t\n\ta1: {m.a3.item()}\tgrad: {m.a3.grad.item() if m.a3.grad else None}")
    print(f"Parameterres before update:\n\tm: {m.m.item()}\tgrad: {m.m.grad.item() if m.m.grad else None}\n\ta2: {m.c.item()}\tgrad: {m.c.grad.item() if m.c.grad else None}")

    # Backpropagate
    loss.backward()
    
    # Update Parameters (m = m - lr*(dL/dm))
    opt.step()
    
#     print(f"Parametrs after update:\n\ta1: {m.a1.item()}\tgrad: {m.a1.grad.item() if m.a1.grad else None}\n\ta2: {m.a2.item()}\tgrad: {m.a2.grad.item() if m.a2.grad else None}\n\t\n\ta1: {m.a3.item()}\tgrad: {m.a3.grad.item() if m.a3.grad else None}")
    print(f"Parameters before update:\n\tm: {m.m.item()}\tgrad: {m.m.grad.item() if m.m.grad else None}\n\ta2: {m.c.item()}\tgrad: {m.c.grad.item() if m.c.grad else None}")
    with torch.no_grad():
        ...
        print(loss.item())
#         ax1.set_xlim(xlim[0], xlim[1])
#         ax1.set_ylim(ylim[0], ylim[1])
        viz_all(model=m, inputs=_X, outputs=_Y, ax=ax1)
#         viz_all(model=m, inputs=_X, outputs=_Y)
        display.display(plt.gcf())

    cmd = input('$: ')
    if cmd.lower().strip() in ['quit', 'q', 'exit', 'stop']:
        break

    plt.cla()
    display.clear_output(wait =True)
    losses.append(loss.item())
        
    if loss.item() < 0.001:
        print('Model fully converged. Stopping.')
        viz_all(model=m, inputs=X, outputs=Y)
        break